In [ ]:
#https://gis.stackexchange.com/questions/151339/rasterize-a-shapefile-with-geopandas-or-fiona-python
# use env:gdal_env_2.7v.2
import geopandas as gpd
import rasterio
from rasterio import features
import numpy as np

Set up your filenames

In [2]:
shp_fn = r'C:/Users/mmann/Documents/ts-raster-example-data/fire17_1/fire17_1.shp'
rst_fn = r'C:/Users/mmann/Documents/ts-raster-example-data/fire17_1/example.tif'
out_fn =  r'C:/Users/mmann/Documents/ts-raster-example-data/fire17_1/AnnualRasters/fire_1980.tif'

Open the file with GeoPANDAS read_file

In [3]:
fires = gpd.read_file(shp_fn)

CHeck and fix datatype issues

In [ ]:
#print fires.columns
#fires['YEAR_'].head()
#fires.YEAR_.unique()

In [4]:
print fires.dtypes.head()

YEAR_        object
STATE        object
AGENCY       object
UNIT_ID      object
FIRE_NAME    object
dtype: object


In [6]:
# subset to year and convert to integer
fires = fires[fires['YEAR_'].isin(['1980'])]
fires['YEAR_'] = fires['YEAR_'].astype(str).astype(int)
print fires.dtypes.head()

YEAR_         int32
STATE        object
AGENCY       object
UNIT_ID      object
FIRE_NAME    object
dtype: object


Open the raster file you want to use as a template for feature burning using rasterio


In [7]:
rst = rasterio.open(rst_fn)
rst.bounds

BoundingBox(left=-374495.83635354, bottom=-616363.3341887, right=566504.16364646, top=592636.6658113)

copy and update the metadata from the input raster for the output


In [8]:
meta = rst.meta.copy()
meta.update(compress='lzw')
rst.close()

Now burn the features into the raster and write it out


The overall idea is to create an iterable containing tuples of (geometry, value), where the geometry is a shapely geometry and the value is what you want to burn into the raster at that geometry's location. Both Fiona and GeoPANDAS use shapely geometries so you are in luck there. In this example a generator is used to iterate through the (geometry,value) pairs which were extracted from the GeoDataFrame and joined together using zip().

## Working

In [44]:
path = r'C:/Users/mmann/Documents/ts-raster-example-data/fire17_1/aet-198403.tif'
output = r'C:/Users/mmann/Documents/ts-raster-example-data/fire17_1/aet-198401-out.tif'

with rasterio.open(path) as src:
    array = src.read()
    profile = src.profile
    profile.update(dtype=rasterio.float32, count=1, compress='lzw',nodata=0)
    out_arr = src.read(1)
    print profile
    
# Write to tif, using the same profile as the source
with rasterio.open(output, 'w', **profile) as dst:
        
    # Write the product as a raster band to a new  file. For
    # the new file's profile, we start with the meta attributes of
    # the source file, but then change the band count to 1, set the
    # dtype to float, and specify LZW compression, missing = 0.

    # this is where we create a generator of geom, value pairs to use in rasterizing
    shapes = ((geom,value) for geom, value in zip(fires.geometry, fires.YEAR_))

    #rasterize shapes 
    burned_value = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=dst.transform)
    #print burned_value.shape
    dst.write(burned_value,1)

{'count': 1, 'crs': CRS({u'lon_0': -120, u'ellps': u'GRS80', u'y_0': -4000000, u'no_defs': True, u'proj': u'aea', u'x_0': 0, u'units': u'm', u'towgs84': u'0,0,0,0,0,0,0', u'lat_2': 40.5, u'lat_1': 34, u'lat_0': 0}), 'interleave': 'band', 'dtype': 'float32', 'driver': u'GTiff', 'transform': Affine(1000.0, 0.0, -374495.83635354,
       0.0, -1000.0, 592636.6658113), 'height': 1209, 'width': 941, 'tiled': False, 'nodata': 0, 'compress': 'lzw'}


## working: spatial filter

In [ ]:
import rasterio
from scipy.signal import medfilt

path = r'C:/Users/mmann/Documents/ts-raster-example-data/fire17_1/aet-198403.tif'
output = "C:/Users/mmann/Documents/ts-raster-example-data/fire17_1/filtered.tif"

with rasterio.open(path) as src:
    array = src.read()
    profile = src.profile

# apply a 5x5 median filter to each band
filtered = medfilt(array, (1, 21, 21)).astype('float32')

# Write to tif, using the same profile as the source
with rasterio.open(output, 'w', **profile) as dst:
    dst.write(filtered)
